In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [3]:
df = pd.read_excel('/content/drive/MyDrive/Churn_pred/customer_churn_large_dataset.xlsx')

In [4]:
df_clean = df.drop(['CustomerID', 'Name'], axis=1)

In [5]:
labelencoder = LabelEncoder()
df_clean['Gender'] = labelencoder.fit_transform(df_clean['Gender'])
df_clean['Location'] = labelencoder.fit_transform(df_clean['Location'])

In [6]:
df_clean['Bill_to_Usage_Ratio'] = df_clean['Monthly_Bill'] / df_clean['Total_Usage_GB']
df_clean['Age_x_Subscription_Length'] = df_clean['Age'] * df_clean['Subscription_Length_Months']

In [7]:
X = df_clean.drop('Churn', axis=1).values
y = df_clean['Churn'].values

In [37]:
class_weights = compute_class_weight('balanced', classes=[0, 1], y=y)
class_weights = torch.FloatTensor(class_weights).cuda()

In [41]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [42]:
X_train_tensor = torch.FloatTensor(X_train).cuda()
y_train_tensor = torch.LongTensor(y_train).cuda()
X_val_tensor = torch.FloatTensor(X_val).cuda()
y_val_tensor = torch.LongTensor(y_val).cuda()
X_test_tensor = torch.FloatTensor(X_test).cuda()
y_test_tensor = torch.LongTensor(y_test).cuda()

In [43]:
class DenserNeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(DenserNeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_dim, 256)
        self.dropout1 = nn.Dropout(0.5)
        self.layer2 = nn.Linear(256, 512)
        self.dropout2 = nn.Dropout(0.5)
        self.layer3 = nn.Linear(512, 256)
        self.dropout3 = nn.Dropout(0.5)
        self.layer4 = nn.Linear(256, 128)
        self.dropout4 = nn.Dropout(0.5)
        self.layer5 = nn.Linear(128, 64)
        self.layer6 = nn.Linear(64, 2)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.dropout1(x)
        x = torch.relu(self.layer2(x))
        x = self.dropout2(x)
        x = torch.relu(self.layer3(x))
        x = self.dropout3(x)
        x = torch.relu(self.layer4(x))
        x = self.dropout4(x)
        x = torch.relu(self.layer5(x))
        x = self.layer6(x)
        return x

In [44]:
input_dim = X_train.shape[1]
model = DenserNeuralNetwork(input_dim).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [51]:
epochs = 50
best_val_loss = float("inf")
epochs_without_improvement = 0
patience = 10

In [52]:
for epoch in range(epochs):
    epoch_loss = 0
    for i, (X_batch, y_batch) in enumerate(train_loader):
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()

    # Calculate validation loss
    model.eval()
    with torch.no_grad():
        y_val_pred = model(X_val_tensor)
        val_loss = criterion(y_val_pred, y_val_tensor).item()
    model.train()

    print(f"Epoch {epoch+1}, Training Loss: {epoch_loss / len(train_loader)}, Validation Loss: {val_loss}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print("Early stopping")
            break

Epoch 1, Training Loss: 0.6931605703671774, Validation Loss: 0.6931496858596802
Epoch 2, Training Loss: 0.6931567743619284, Validation Loss: 0.6931447386741638
Epoch 3, Training Loss: 0.6931619867642721, Validation Loss: 0.6931461691856384
Epoch 4, Training Loss: 0.6931528467814128, Validation Loss: 0.6931507587432861
Epoch 5, Training Loss: 0.693128646850586, Validation Loss: 0.6931462287902832
Epoch 6, Training Loss: 0.6932224153518677, Validation Loss: 0.6931470632553101
Epoch 7, Training Loss: 0.6932021759033203, Validation Loss: 0.693147599697113
Epoch 8, Training Loss: 0.6931641912142436, Validation Loss: 0.6931463479995728
Epoch 9, Training Loss: 0.6931635188102723, Validation Loss: 0.6931459307670593
Epoch 10, Training Loss: 0.6931550856272379, Validation Loss: 0.6931508183479309
Epoch 11, Training Loss: 0.6931764337857564, Validation Loss: 0.6931418776512146
Epoch 12, Training Loss: 0.693159530544281, Validation Loss: 0.6931437253952026
Epoch 13, Training Loss: 0.6931546361287

In [53]:
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    _, y_pred = torch.max(y_pred_tensor, dim=1)

In [54]:
y_pred = y_pred.cpu().numpy()
y_test = y_test_tensor.cpu().numpy()

In [55]:
accuracy_nn = accuracy_score(y_test, y_pred)
f1_nn = f1_score(y_test, y_pred)
precision_nn = precision_score(y_test, y_pred)
recall_nn = recall_score(y_test, y_pred)

In [56]:
print("Dense Neural Network - Accuracy:", accuracy_nn, "F1:", f1_nn, "Precision:", precision_nn, "Recall:", recall_nn)

Dense Neural Network - Accuracy: 0.4978 F1: 0.6647082387501669 Precision: 0.4978 Recall: 1.0
